# Hands On with Darts (Time Series Analysis)

In [2]:
!pip -q install darts

## Synthetic Data

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [4]:
start_date = datetime.now()
num_days = 365

dates = [start_date + timedelta(days=i) for i in range(num_days)]
sales = [100 + i * 10 + np.random.normal(0, 10) for i in range(num_days)]

data = pd.DataFrame({
    'date': dates,
    'sales': sales
})

In [6]:
from darts import TimeSeries

series = TimeSeries.from_dataframe(data, "date", "sales")

## Darts Models

In [19]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


### Simple Model (Exponential Smoothing)

In [7]:
from darts.models import ExponentialSmoothing

model = ExponentialSmoothing()
model.fit(series)
prediction = model.predict(90)

Complex Model (RNN)

In [45]:
from sklearn.preprocessing import MinMaxScaler
from darts.dataprocessing.transformers import Scaler

scaler = MinMaxScaler(feature_range=(-0.5, 0.5))

transformer = Scaler(scaler)
series_transformed = transformer.fit_transform(series)

In [50]:
from darts.models import RNNModel

rnn_model = RNNModel(
    model="LSTM",
    input_chunk_length=90,
    training_length=180,
    hidden_dim=50,
    n_rnn_layers=2,
    dropout=0.2,
    force_reset=True,
    show_warnings=True,
    pl_trainer_kwargs={
        "accelerator": "gpu",
        "devices": [0],
    }
)
rnn_model.fit(series_transformed)
prediction = rnn_model.predict(90)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | GRU              | 23.2 K
4 | V             | Linear           | 51    
---------------------------------------------------
23.3 K    Trainable params
0         Non-trainable params
23.3 K    Total params
0.093     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [52]:
prediction_untransformed = transformer.inverse_transform(prediction)